In [1]:
import pandas as pd
import requests
from ast import literal_eval
from tqdm import tqdm

In [2]:
df=pd.read_csv('PMC_OpenAlex.csv',converters={"authors": lambda x: x.strip("[]").replace("\"","").split(", ")})
#df['authors']=df['authors'].apply(literal_eval)

In [3]:
listAuthors=sum(list(df['authors']),[])

In [4]:
listAuthors=list(set([i.replace("\'","") for i in listAuthors]))

In [5]:
len(listAuthors)

38515

In [6]:
listAuthors=listAuthors[1:]

In [ ]:
outfile='publications_authors_PMC.csv'

urlbase='https://api.openalex.org/works?filter=authorships.author.id:'
final='&mailto=floriana.gargiulo@cnrs.fr&per-page=100&page='

allDataFrame=pd.DataFrame()

for au in tqdm(listAuthors[3080:]):
    #au='A2142515187'
    
    ID=[]
    DOI=[]
    PMID=[]
    TITLE=[]
    DATE=[]
    ISRETRACTED=[]
    JOURN_ID=[]
    TYPE=[]
    AUTHORS=[]
    NCITATIONS=[]
    CITYEAR=[]
    MESH=[]
    
    url=urlbase+au+final+str(1)
    cc=requests.get(url).json()
    if cc['meta']['count']<1000:
        page=1
        while True:
            url=urlbase+au+final+str(page)
            cc=requests.get(url).json()
            for paper in cc['results']:
                try:
                    ID.append(paper['id'])
                except:
                    ID.append('NONE')
                    
                try:
                    DOI.append(paper['doi'])
                except:
                    DOI.append('NONE')

                try:
                    PMID.append(paper['ids']['pmid'])
                except:
                    PMID.append('NONE')

                try:
                    TITLE.append(paper['title'])
                except:
                    TITLE.append('NONE')

                try:
                    DATE.append(paper['publication_date'])
                except:
                    DATE.append('NONE')

                try:
                    ISRETRACTED.append(paper['is_retracted'])
                except:
                    ISRETRACTED.append('NONE')

                try:
                    JOURN_ID.append(paper['host_venue']['id'])
                except:
                    JOURN_ID.append('NONE')

                try:
                    TYPE.append(paper['type'])
                except:
                    TYPE.append('NONE')

                try:
                    AUTHORS.append([au['author']['id'].replace('https://openalex.org/','') for au in paper['authorships']])
                except:
                    AUTHORS.append('NONE')

                try:
                    NCITATIONS.append(paper['cited_by_count'])
                except:
                    NCITATIONS.append('NONE')

                try:
                    CITYEAR.append([(item['year'],item['cited_by_count']) for item in paper['counts_by_year']])
                except:
                    CITYEAR.append('NONE')

                try:
                    MESH.append(paper['mesh'])
                except:
                    MESH.append('NONE')

            if cc['meta']['count']<page*100:
                break
            page=page+1


        thisDataFrame=pd.DataFrame()
        thisDataFrame['QUERY_AUTHOR']=[au for k in range(len(ID))]
        thisDataFrame['ID']= ID
        thisDataFrame['DOI']=DOI
        thisDataFrame['PMID']=PMID
        thisDataFrame['TITLE']=TITLE
        thisDataFrame['DATE']=DATE
        thisDataFrame['ISRETRACTED']=ISRETRACTED
        thisDataFrame['JOURN_ID']=JOURN_ID
        thisDataFrame['TYPE']=TYPE
        thisDataFrame['AUTHORS']=AUTHORS
        thisDataFrame['NCITATIONS']=NCITATIONS
        thisDataFrame['CITYEAR']=CITYEAR
        thisDataFrame['MESH']=MESH

        #allDataFrame=pd.concat([allDataFrame,thisDataFrame])
        thisDataFrame.to_csv(outfile, mode='a', index=False, header=False)


  0%|                                     | 56/35434 [05:56<70:55:39,  7.22s/it]

In [8]:
cc

{'meta': {'count': 232, 'db_response_time_ms': 20, 'page': 1, 'per_page': 100},
 'results': [{'id': 'https://openalex.org/W2004228538',
   'doi': 'https://doi.org/10.1006/jmbi.2001.4762',
   'title': 'FUGUE: sequence-structure homology recognition using environment-specific substitution tables and structure-dependent gap penalties11Edited by B. Honig',
   'display_name': 'FUGUE: sequence-structure homology recognition using environment-specific substitution tables and structure-dependent gap penalties11Edited by B. Honig',
   'publication_year': 2001,
   'publication_date': '2001-06-29',
   'ids': {'openalex': 'https://openalex.org/W2004228538',
    'doi': 'https://doi.org/10.1006/jmbi.2001.4762',
    'mag': '2004228538',
    'pmid': 'https://pubmed.ncbi.nlm.nih.gov/11419950'},
   'host_venue': {'id': 'https://openalex.org/V46058472',
    'issn_l': '0022-2836',
    'issn': ['1089-8638', '0022-2836'],
    'display_name': 'Journal of Molecular Biology',
    'publisher': 'Elsevier',
    '